# ICAIF 2024 Finance-RAG Challenge Baseline

This notebook demonstrates a **baseline solution** for the **ICAIF 2024 Finance-RAG Challenge**. The goal of the challenge is to create a **Retrieval-Augmented Generation (RAG)** system for financial data. Participants are tasked with developing systems that retrieve relevant documents from a large corpus and provide accurate, context-aware responses to user queries.

---

## System Components

The system is divided into two main components:

1. **Retrieval**: Retrieves relevant financial documents from a large corpus based on a user query.
2. **Reranking**: Refines the ranking of the retrieved documents to ensure the most relevant information is prioritized.

---

## Model Overview

This baseline notebook uses a combination of `SentenceTransformer` and `CrossEncoder` models to perform these tasks:

- The **retrieval model** is responsible for encoding both the queries and documents into embeddings.
- The **reranking model** evaluates the relevance of the retrieved documents and reorders them accordingly.

In this example, the baseline task used is **FinDER**, which is one of the seven available tasks in the FinanceRAG project. The retrieval model used is `intfloat/e5-large-v2`, and the reranking is performed using `cross-encoder/ms-marco-MiniLM-L-12-v2`. Both of these models can be substituted with other models supported by the `sentence_transformers` library for performance experimentation.

---

## Goal

The goal of this notebook is to provide a **solid foundation** for participants to build more advanced solutions for the challenge. Feel free to customize the task, retrieval model, and reranking model as needed.

---

## Repository Setup and Environment Configuration

You can find the repository for this project on GitHub [here](https://github.com/JiH00nKw0n/FinanceRAG).

To clone the repository and set up the environment, follow these steps:

### 1. Clone the repository:

```bash
git clone https://github.com/linq-rag/FinanceRAG.git
cd FinanceRAG
```

### 2. Set up the Python environment:

#### If using `venv` (Python 3.11 or higher required):

```bash
python3 -m venv .venv
source .venv/bin/activate  # On Windows use .venv\Scriptsctivate
pip install --upgrade pip
pip install -r requirements.txt
```

#### If using `conda`:

```bash
conda create -n financerag python=3.11
conda activate financerag
pip install -r requirements.txt
```

You should now be ready to run the baseline notebook!

In [1]:
# Step 1: Import necessary libraries
# --------------------------------------
# Import required libraries for document retrieval, reranking, and logging setup.
from sentence_transformers import CrossEncoder
import logging

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder
from financerag.tasks import FinDER

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)


/home/s113062513/miniconda3/envs/financerag/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Step 2: Initialize FinDER Task
# --------------------------
# In this baseline example, we are using the FinDER task, one of the seven available tasks in this project.
# If you want to use a different task, for example, 'OtherTask', you can change the task initialization as follows:
#
# Example:
# from financerag.tasks import OtherTask
# finder_task = OtherTask()
#
# For this baseline, we proceed with FinDER.
finder_task = FinDER()


INFO:financerag.common.loader:Loading Corpus...
INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsi

In [3]:
# Step 3: Initialize DenseRetriever model
# -------------------------------------
# Initialize the retrieval model using SentenceTransformers. This model will be responsible
# for encoding both the queries and documents into embeddings.
#
# You can replace 'intfloat/e5-large-v2' with any other model supported by SentenceTransformers.
# For example: 'BAAI/bge-large-en-v1.5', 'Linq-AI-Research/Linq-Embed-Mistral', etc.
encoder_model = SentenceTransformerEncoder(
    model_name_or_path='intfloat/e5-large-v2',
    query_prompt='query: ',
    doc_prompt='passage: ',
)

retrieval_model = DenseRetrieval(
    model=encoder_model
)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/e5-large-v2


In [4]:
# Step 4: Perform retrieval
# ---------------------
# Use the model to retrieve relevant documents for given queries.
retrieval_model = DenseRetrieval(
    model=encoder_model
)

retrieval_result = finder_task.retrieve(
    retriever=retrieval_model
)

# Print a portion of the retrieval results to verify the output.
print(f"Retrieved results for {len(retrieval_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in retrieval_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query


INFO:financerag.retrieval.dense:Encoding queries...
Batches: 100%|██████████| 4/4 [00:00<00:00, 20.84it/s]
INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...
Batches: 100%|██████████| 217/217 [01:05<00:00,  3.33it/s]


Retrieved results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230966, Score = 0.8739963173866272
  Document 2: Document ID = MSFT20230216, Score = 0.8645690083503723
  Document 3: Document ID = MSFT20230015, Score = 0.8594436645507812
  Document 4: Document ID = MSFT20230254, Score = 0.8580108880996704
  Document 5: Document ID = MSFT20230155, Score = 0.8534094095230103


In [5]:
# Step 5: Initialize CrossEncoder Reranker
# --------------------------------------
# The CrossEncoder model will be used to rerank the retrieved documents based on relevance.
#
# You can replace 'cross-encoder/ms-marco-MiniLM-L-12-v2' with any other model supported by CrossEncoder.
# For example: 'cross-encoder/ms-marco-TinyBERT-L-2', 'cross-encoder/stsb-roberta-large', etc.
reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
)


INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [6]:
# Step 6: Perform reranking
# -------------------------
# Rerank the top 100 retrieved documents using the CrossEncoder model.
reranking_result = finder_task.rerank(
    reranker=reranker,
    results=retrieval_result,
    top_k=100,  # Rerank the top 100 documents
    batch_size=32
)

# Print a portion of the reranking results to verify the output.
print(f"Reranking results for {len(reranking_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in reranking_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....
Batches: 100%|██████████| 675/675 [00:25<00:00, 26.66it/s]


Reranking results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230254, Score = 6.622737884521484
  Document 2: Document ID = MSFT20230233, Score = 6.0889434814453125
  Document 3: Document ID = MSFT20230230, Score = 5.898367404937744
  Document 4: Document ID = MSFT20230236, Score = 5.747088432312012
  Document 5: Document ID = MSFT20230216, Score = 5.488572120666504


In [7]:
# Step 7: Save results
# -------------------
# Save the results to the specified output directory as a CSV file.
output_dir = './results'
finder_task.save_results(output_dir=output_dir)

# Confirm the results have been saved.
print(f"Results have been saved to {output_dir}/FinDER/results.csv")


INFO:financerag.tasks.BaseTask:Output directory set to: ./results/FinDER
INFO:financerag.tasks.BaseTask:Saving top 10 results to CSV file: ./results/FinDER/results.csv
INFO:financerag.tasks.BaseTask:Writing header ['query_id', 'corpus_id'] to CSV.
INFO:financerag.tasks.BaseTask:Top 10 results saved successfully to ./results/FinDER/results.csv


Results have been saved to ./results/FinDER/results.csv
